# 模型优化------剪枝
参考[深度学习中的网络剪枝（pruning）简介](https://www.jianshu.com/p/76885bef9671)
[Model Pruning in Deep Learning](https://towardsdatascience.com/scooping-into-model-pruning-in-deep-learning-da92217b84ac)
[Keras Model pruning Exploration](https://colab.research.google.com/github/matthew-mcateer/Keras_pruning/blob/master/Model_pruning_exploration.ipynb)
剪枝是一种神经网络优化技术，他通过删除网络中不必要的权重，使网络在可以接受的精度下降范围内获得更快的运行速度、更小的延时。剪枝后的模型可以获得更小的压缩体积，便于部署到边缘设备上。

剪枝是近年来的研究热点，目前已经有很多的剪枝策略，工业界常用的是"基于权重重要性的剪枝"（Magnitude-base Pruning）。

"Magnitude-base Pruning"通过将不重要的weights（biases）设置为0来实现剪枝，加快网络的运行速度。剪枝后需要对网络进行训练，从而保证网络剪枝后的精度（如果想要进一步缩小模型的体积可以再进行训练后量化操作）。
可以对一个预训练的网络剪枝再训练，也可以对一个随机初始化的网络剪枝再训练。一般而言前者可以取得更好的效果，因此一般剪枝都是对预训练好的网络进行。

## Magnitude-base Pruning工作过程简介：

### 首先我们需要弄明白何为重要的权重，何为不重要的权重
在梯度下降的优化过程中，并非所有权重都使用相同的梯度幅度进行更新。某些权重将使用比其他权重更大的梯度幅度进行更新。优化器认为这些权重是很重要的，他们可以最大程度的减小训练目标（loss）。其他的权重认为是不重要的。

### 工作过程：
**1. 剪枝**
在实践中选取幅值大的权重作为重要权重(参考[深度学习中的网络剪枝（pruning）简介](https://www.jianshu.com/p/76885bef9671)），并以此为基础完成剪枝：
譬如我们要减掉30%的权重（sparsity_percentage=30%的Magnitude-base Pruning），对于每一层（譬如dense层）:
+ 提取某一层的权重
+ 计算权重张量的列向量的L2范数
+ 据L2范数进行排序
+ 将低30%的L2范数对应的权重列向量用0代替
*这样剪掉的是不重要的神经元，如果把列向量的L2范数用每一个权重的绝对值代替，那么剪掉的是神经元中的不重要的权重。*

以上剪枝过程每层的sparsity_percentage都是一样的，这是按照层来剪枝。

**2. 训练**
对剪枝后的模型进行训练，训练过程中被修剪掉的权重固定为0不变。

**3. 量化**
对剪枝、训练后的模型执行量化，进一步减小模型的体积，便于在边缘设备上的部署。

### Q&A：
*如何保证最终模型的sparsity与剪枝时保持一致*

**Q:**

**预训练--->剪枝--->fine-tune--->量化**

剪枝fine-tune后的模型在量化阶段如何保证量化后模型的sparsity保持一致，也即权重的实际值r为0时，量化值q也为0？
+ 假如量化为uint8类型[0, 255]，那么权重的最小值应该是0，这该怎么保证呢？
+ 其他量化方式又是如何保证量化后的sparsity保持一致的？

**A:**

这是通常的剪枝后部署网络模型到边缘设备的SOP。对于sparsity保持一致的情况，量化后网络inference时进行稀疏矩阵运算可能加快运算速度（视底层实现）。至于sparsity是否保持一致不清楚。网络上（知乎）也有说法剪枝不一定能保证网络速度加快，但是剪枝后模型可压缩度变大，便于边缘设备部署。tensorflow给出的例子上也是只关注了模型的可压缩度（譬如lenet网络在剪枝后稀疏度为80%，通过zip压缩可以获得3倍的压缩效果，keras转换为tflite后再量化加上之前的剪枝一共可以获得10倍的压缩效果），没有关注模型的inference时间有无改善。

**Q:**

**预训练--->量化--->剪枝**

剪枝前对预训练模型量化，量化后的模型剪枝后直接应用，那么可以保证模型的sparsity与剪枝的期望sparsity一致，只是这样精度是不是损失较大？

**A:**

这种方式一般不使用，这样操作精度掉的会比较多

**Q:**

**预训练--->量化--->剪枝--->retrain**

剪枝前对预训练模型量化，剪枝后的模型进行retrain,量化后的模型可以retrain吗？retrain过程中是得保持sparsity不变吧（0权重不能被更新）？

**A:**

这种方式一般不使用，训练量化后的模型，没有成熟的商用，处于实验室阶段，常见于论文中

### 如何更好的剪枝
*保证剪枝后的精度*

+ 不要修剪分类头
+ 只修剪某几层
+ 尽量修剪后面的层，不要修剪关键层
+ 修剪bias会大幅影响模型的精度，keras的API模型不修剪bias
+ 定义合适的网络模型
+ fine-tune阶段选择合适的learning rate
+ 不要修剪的太频繁，在修剪过程中给模型留出修剪后恢复精度的充裕时间。

### 剪枝示例

*参见magnitude_based_pruning.ipynb*

剪枝的对象为weight(bias不推荐裁剪)，当然你也可以尝试修剪整个unit/neuron等（keras中只找到了tfmot.sparsity.keras.prune_low_magnitude这种剪枝方式，只支持裁剪权重；要想裁剪整个神经元可以参考[Keras Model pruning Exploration](https://colab.research.google.com/github/matthew-mcateer/Keras_pruning/blob/master/Model_pruning_exploration.ipynb#scrollTo=YOIHH60dFiVE) 这里面的剪枝都是手动修改权重实现的，很简单，嘿嘿）。